In [ ]:
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel

In [12]:
class CW2KM:
    def __init__(self, n_clusters, gamma, max_iter=100, p=2):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.max_iter = max_iter
        self.p = p

    def fit(self, V):
        self.K = [rbf_kernel(x, self.gamma) for x in self.X]

        n = self.K[0].shape[0]
        self.U = np.zeros((n, self.n_clusters))
        self.weights = np.full((len(V), self.n_clusters), 1 / len(V))

        for i in range(self.max_iter):
            # Updating the cluster indicator matrix U
            self.U *= 0
            for j in range(n):
                dist = []
                for v in range(len(V)):  
                    for k in range(self.n_clusters):
                        K_ii = np.diag(self.K[v])
                        K_ij = np.sum(self.U[:, k] * self.K[v][:, j])/np.sum(self.U[:, k])
                        # soorat = 0
                        # makhraj = 0
                        # for l in range(0, n):
                        #     for m in range(0, n):
                        #         soorat += self.U[l, k] * self.U[m, k] * self.K[l, m]
                        #         makhraj += self.U[l, k] * self.U[m, k]
                        # K_jj = soorat/makhraj
                        K_jj = np.sum(self.U[:n, k][:, np.newaxis] * self.U[:n, k] * self.K[v])\
                                /np.sum(self.U[:n, k][:, np.newaxis] * self.U[:n, k] * self.K[v])
                        d = K_ii - 2 * K_ij + K_jj
                        dist.append(d)
                # dist = [self.kernel_dist(k, j) for k in range(self.n_clusters)]
                self.U[j, np.argmin(dist)] = 1

            # Updating the cluster weight ω_vk

            # Sum-of-squared errors of the kth cluster in the vth view
            D = np.zeros((len(V), self.n_clusters))
            for j in range(n):
                for v in range(len(V)):  
                    for k in range(self.n_clusters):
                        K_ii = np.diag(self.K[v])
                        K_ij = np.sum(self.U[:, k] * self.K[v][:, j])/np.sum(self.U[:, k])
                        # soorat = 0
                        # makhraj = 0
                        # for l in range(0, n):
                        #     for m in range(0, n):
                        #         soorat += self.U[l, k] * self.U[m, k] * self.K[l, m]
                        #         makhraj += self.U[l, k] * self.U[m, k]
                        # K_jj = soorat/makhraj
                        K_jj = np.sum(self.U[:n, k][:, np.newaxis] * self.U[:n, k] * self.K[v])\
                                /np.sum(self.U[:n, k][:, np.newaxis] * self.U[:n, k] * self.K[v])
                        d = K_ii - 2 * K_ij + K_jj
                        D[v, k] += d

            # Updaing w
            # for k in range(self.n_clusters): 
            #     for v in range(len(V)): 
            #         for v_ in range(len(V)): 
            #             self.weights[v, k] = 1 / ((D[v, k]/D[v_, k]) ** (1/(self.p-1)))

            self.weights = 1 / ((D[:, np.newaxis, :] / D[:, :, k]) ** (1 / (self.p - 1))).sum(axis=1)


In [7]:
import numpy as np

def nested_sum(U, n, k):
    result = 0

    # Perform the nested summation using a nested loop
    for i in range(1, n+1):
        for j in range(1, n+1):
            result += U[i-1, k] * U[j-1, k]

    return result

# Example usage:
n = 3  # The value of n
k = 1  # The value of k
U = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

result = nested_sum(U, n, k)
print("Result:", result)

Result: 225


In [6]:
def nested_sum(U, n, k):
    return np.sum(U[:n, k][:, np.newaxis] * U[:n, k])

# Example usage:
n = 3  # The value of n
k = 1  # The value of k
U = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

result = nested_sum(U, n, k)
print("Result:", result)

Result: 225


In [9]:
import numpy as np

def nested_sum_with_matrix(U, K, n, k):
    return np.sum(U[:n, k][:, np.newaxis] * U[:n, k] * K[:n, :n])

# Example usage:
n = 3  # The value of n
k = 1  # The value of k
U = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
K = np.array([[10, 11, 12],
              [13, 14, 15],
              [16, 17, 18]])

result = nested_sum_with_matrix(U, K, n, k)
print("Result:", result)


ValueError: operands could not be broadcast together with shapes (2,2) (3,3) 